Initial attempt at saving the image from ChromeDino so that we can then try and make use of the game environment

In [1]:
import re
import time
import requests
from bs4 import BeautifulSoup
#from tqdm import tqdm, tnrange
import numpy as np
from collections import defaultdict,Counter
from scipy.misc import imread, imsave

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import base64
import io
from PIL import Image

from matplotlib import pyplot as plt, rcParams
from matplotlib import cm
%matplotlib inline
rcParams["figure.figsize"] = (16,12)    # Use this line to increase your figure size (optional).

In [2]:
def get_cactus_height(x,y, img):
    while np.any(img[x,y-7:y+7]):
        x -= 1
    return x

def find_cacti(img):
    height,width = np.shape(img)
    dino = img[height//2:,:height//2]
    vision = round(height*.8)
    bird_vision = round(height*0.6)
    field = img[vision,height//2 - width//30 - 5:] + img[bird_vision,height//2 - width//30 - 5:]
    distance = []
    cacti_width = []
    cacti_height = []
    prev_pixel = field[0]
    cur_dist = 0
    cur_width = 0
    for i,pixel in enumerate(field):
        if prev_pixel == 0 and pixel != 0:
            distance.append(cur_dist)
            cacti_height.append(get_cactus_height(vision, height//2 - width//30 - 5 + i, img))
            cur_dist = 0
        if prev_pixel != 0 and pixel == 0:
            cacti_width.append(cur_width)
            cur_width = 0
        if pixel == 0:
            cur_dist += 1
        else:
            cur_width += 1
        prev_pixel = pixel
    cacti_width = cacti_width[1:]
    return distance, cacti_width, cacti_height

In [ ]:
js_call = "return document.getElementsByClassName('runner-canvas')[0].toDataURL('image/png').substring(21);"
second_js = "Runner.instance_.tRex.startJump({})"
cur_speed_js = "return Runner.instance_.currentSpeed"
url = "https://chromedino.com/"
browser = webdriver.Chrome()
try:
    browser.get(url)

    body = browser.find_element_by_id('t')
    body.send_keys(Keys.ARROW_UP)
    prev_dist = None
    prev_width = None
    values = []
    speed_factor = 0.999
    while True:
        time.sleep(.0015)
        speed_factor += .0001
        if speed_factor > 2.5:
            speed_factor = 2.5
        b64_image = browser.execute_script(js_call)
        image = base64.b64decode(b64_image)
        buf = io.BytesIO(image)
        img = Image.open(buf).convert('LA')
        np_img = np.array(img)[:,:,0]
        distances, widths, heights = find_cacti(np_img)
        #print(distances, heights)
        #print(distances,widths)
        if len(values) > 70:
            if values[-1][0] == distances and values[-2][0] == distances and values[-3][0] == distances and values[-20][0] == distances:
                pass
                #print("GAME OVER")
                #cur_speed = float(browser.execute_script(cur_speed_js))
                #browser.execute_script(second_js.format(cur_speed*3))
                #values.append((-1,"NEW"))
            elif len(distances) != 0:
                distances_copy = distances.copy()
                for i,v in enumerate(distances_copy):
                    try:
                        if abs(distances_copy[i+1] - v) < 10:
                            distances.remove(v)
                        else:
                            break
                    except:
                        pass
                
                if 85 <= heights[0] <= 95:
                    if distances[0] < (102 * speed_factor):
                        values.append((distances,widths,"J"))
                        cur_speed = float(browser.execute_script(cur_speed_js))
                        browser.execute_script(second_js.format(cur_speed*3))
                        print("SF", speed_factor)
                if 95 <= heights[0] <= 103:
                    if distances[0] < (113 * speed_factor):
                        values.append((distances,widths,"J"))
                        cur_speed = float(browser.execute_script(cur_speed_js))
                        browser.execute_script(second_js.format(cur_speed*3))
                        print("SF", speed_factor)
                if 104 <= heights[0]:
                    if distances[0] < (116 * speed_factor):
                        values.append((distances,widths,"J"))
                        cur_speed = float(browser.execute_script(cur_speed_js))
                        browser.execute_script(second_js.format(cur_speed*3))
                        print("SF", speed_factor)
            else:
                values.append((distances,widths,"N"))
        else:
            values.append((distances,widths,"N"))
        prev_dist, prev_width = distances, widths
except Exception as e:
    print("Errored out at step 1", e)
finally:
    input()
    browser.close() #make sure we close it when we are finished.

SF 1.0404999999999955
SF 1.0405999999999955
SF 1.0406999999999955
SF 1.0407999999999955
SF 1.0408999999999955
SF 1.0409999999999955
SF 1.0410999999999955
SF 1.0411999999999955
SF 1.0412999999999955
SF 1.0413999999999954
SF 1.0414999999999954
SF 1.0415999999999954
SF 1.0416999999999954
SF 1.0417999999999954
SF 1.0418999999999954
SF 1.0419999999999954
SF 1.044899999999995
SF 1.044999999999995
SF 1.045099999999995
SF 1.045199999999995
SF 1.045299999999995
SF 1.045399999999995
SF 1.045499999999995
SF 1.045599999999995
SF 1.045699999999995
SF 1.045799999999995
SF 1.045899999999995
SF 1.045999999999995
SF 1.046099999999995
SF 1.046199999999995
SF 1.046299999999995
SF 1.046399999999995
SF 1.0501999999999945
SF 1.0502999999999945
SF 1.0503999999999944
SF 1.0504999999999944
SF 1.0505999999999944
SF 1.0506999999999944
SF 1.0507999999999944
SF 1.0508999999999944
SF 1.0509999999999944
SF 1.0510999999999944
SF 1.0511999999999944
SF 1.0512999999999944
SF 1.0513999999999943
SF 1.0514999999999943
SF 1